In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
from matplotlib import pyplot as plt

from micro_price_trading import Preprocess, TwoAssetSimulation
from micro_price_trading.config import TWENTY_SECOND_DAY, DATA_PATH

In [ ]:
raw = Preprocess('Test_TBT_TBF_data.csv')
data = raw.process()

In [ ]:
## Sample input data

n = 10
input_data = np.zeros((n, 1171, 3))
sim = TwoAssetSimulation(data, steps=TWENTY_SECOND_DAY, seed=0)

for i in trange(10):
    input_data[i,:,:] = sim.states
    sim._reset_simulation()


In [ ]:
q_values = pd.read_csv(DATA_PATH.joinpath('q_values_9_27.csv'), index_col=0)

In [ ]:
def base_twap(t, asset):
    
    indices_to_buy_at = [idx for idx in range(0, TWENTY_SECOND_DAY, t)]
    asset_to_buy = asset
    
    avg_prices = input_data[:,indices_to_buy_at, asset_to_buy].mean(axis = 1)
    
    return avg_prices

In [ ]:
base_twap(10, 1), base_twap(10, 2)

In [ ]:
def random_twap(t, weights=(0.5, 0.5)):
    
    random_numbers = np.random.rand(len([idx for idx in range(0, TWENTY_SECOND_DAY, 10)]))
    indices_to_buy_asset_1 = np.where(random_numbers >= weights[0])[0]
    indices_to_buy_asset_2 = np.where(random_numbers <= weights[0])[0]
    
    avg_prices_1 = input_data[:,indices_to_buy_asset_1, 1].mean(axis = 1)
    avg_prices_2 = input_data[:,indices_to_buy_asset_2, 1].mean(axis = 1)
    
    return avg_prices_1, avg_prices_2

In [ ]:
random_twap(10, weights=(0.5, 0.5))

In [ ]:
def optimal_twap(t, iters, q_vals):
    sim = TwoAssetSimulation(data, steps=TWENTY_SECOND_DAY, seed=0)
    shares1 = list()
    shares2 = list()
    buy1 = list()
    buy2 = list()
    prices1 = list()
    prices2 = list()
    for _ in trange(iters):
        shares1.append([0])
        shares2.append([0])
        prices1.append([])
        prices2.append([])
        buy1.append([])
        buy2.append([])
        for idx in range(0, TWENTY_SECOND_DAY, t):
            res_imb_state = sim.states.states[idx]
            qs = q_vals.loc[res_imb_state]
            if qs[0] > qs[1]:
                asset = 1
            else:
                asset = 2
            shares = 1 if asset == 2 else 2
            if asset == 1:
                shares1[-1].append(shares1[-1][-1]+shares)
                shares2[-1].append(shares2[-1][-1])
                prices1[-1].append(sim.states[f'mid_{asset}'][idx])
                buy1[-1].append(shares)
            else:
                shares2[-1].append(shares2[-1][-1]+shares)
                shares1[-1].append(shares1[-1][-1])
                prices2[-1].append(sim.states[f'mid_{asset}'][idx])
                buy2[-1].append(shares)
        sim._reset_simulation()
    return prices1, prices2, buy1, buy2, np.array(shares1), np.array(shares2)

In [ ]:
prices1, shares1 = base_twap(3, 1, 1000)

In [ ]:
prices2, shares2 = base_twap(3, 2, 1000)

In [ ]:
p1, p2, b1, b2, s1, s2 = random_twap(3, 1000, (0.5, 0.5))

In [ ]:
p1_means = [np.mean(l) for l in p1]
p2_means = [np.mean(l) for l in p2]

In [ ]:
avg1 = (p1_means-prices1.mean(axis=1))*s1[:, -1]

In [ ]:
avg2 = (p2_means-prices2.mean(axis=1))*s2[:, -1]

In [ ]:
diff = avg1 + avg2

In [ ]:
(diff.mean() - 1.96 * diff.std()/np.sqrt(len(diff)), diff.mean() + 1.96 * diff.std()/np.sqrt(len(diff)))

In [ ]:
op1, op2, ob1, ob2, os1, os2 = optimal_twap(3, 1000, q_values)

In [ ]:
op1_means = [np.mean(l) for l in op1]
op2_means = [np.mean(l) for l in op2]

In [ ]:
oavg1 = (prices1.mean(axis=1)-op1_means)*os1[:, -1]

In [ ]:
oavg2 = (prices2.mean(axis=1)-op2_means)*os2[:, -1]

In [ ]:
odiff = oavg1 + oavg2

In [ ]:
(odiff.mean() - 1.44 * odiff.std()/np.sqrt(len(odiff)), odiff.mean() + 1.44 * odiff.std()/np.sqrt(len(odiff)))

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
shares1.plot('asset2', 'asset1', kind='line', ax=ax, label='TWAP for TBF')
shares2.plot('asset2', 'asset1', kind='line', ax=ax, label='TWAP for TBT')
ax.plot([shares2.asset2.iloc[-1], 0], [0, shares1.asset1.iloc[-1]], 'lime', linewidth=2)
ax.plot([shares2.asset2.iloc[-1], 0], [0, shares1.asset1.iloc[-1]], 'k--', linewidth=2, dashes=(3, 3))
ax.set_xlim(-5, shares1.asset1.max()+10)
for idx in range(s1.shape[0]):
    ax.plot(s2[idx], s1[idx], 'm', alpha=0.7)

ax.plot(s2[idx], s1[idx], 'm', alpha=0.7, label='Random Walk')
for idx in range(os1.shape[0]):
    ax.plot(os2[idx], os1[idx], 'c', alpha=0.7)
ax.plot(os2[idx], os1[idx], 'c', alpha=0.7, label='Q-Value Strategy')
ax.legend()
ax.set_xlabel('Shares of TBT')
ax.set_ylabel('Shares of TBF')
plt.suptitle('Shares of TBF vs. TBT with Different Execution Algorithms', fontsize=20)
plt.title(f'85% CI over TWAP for individual assets: {odiff.mean(): .3f}'+u' \u00B1 '+f'{1.44 * odiff.std()/np.sqrt(len(odiff)):.3f} ({odiff.mean() - 1.44 * odiff.std()/np.sqrt(len(odiff)): .3f}, {odiff.mean() + 1.44 * odiff.std()/np.sqrt(len(odiff)): .3f})')

## TWAP - 5 minutes

In [ ]:
prices15, shares15 = base_twap(15, 1, 500)

In [ ]:
prices25, shares25 = base_twap(15, 2, 500)

In [ ]:
p15, p25, b15, b25, s15, s25 = random_twap(15, 500, (0.5, 0.5))

In [ ]:
p15_means = [np.mean(l) for l in p15]
p25_means = [np.mean(l) for l in p25]

In [ ]:
avg15 = (prices15.mean(axis=1)-p15_means)*s15[:, -1]

In [ ]:
avg25 = (prices25.mean(axis=1)-p25_means)*s25[:, -1]

In [ ]:
diff5 = avg15 + avg25

In [ ]:
(diff5.mean() - 1.96 * diff5.std()/np.sqrt(len(diff5)), diff5.mean() + 1.96 * diff5.std()/np.sqrt(len(diff5)))

In [ ]:
op15, op25, ob15, ob25, os15, os25 = optimal_twap(15, 500, q_values)

In [ ]:
op15_means = [np.mean(l) for l in op15]
op25_means = [np.mean(l) for l in op25]

In [ ]:
oavg15 = (prices15.mean(axis=1)-op15_means)*os15[:, -1]

In [ ]:
oavg25 = (prices25.mean(axis=1)-op25_means)*os25[:, -1]

In [ ]:
odiff5 = oavg15 + oavg25

In [ ]:
(odiff5.mean() - 1.96 * odiff5.std()/np.sqrt(len(odiff5)), odiff5.mean() + 1.96 * odiff5.std()/np.sqrt(len(odiff5)))

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
shares15.plot('asset2', 'asset1', kind='line', ax=ax, label='asset1')
shares25.plot('asset2', 'asset1', kind='line', ax=ax, label='asset2')
ax.plot([shares25.asset2.iloc[-1], 0], [0, shares15.asset1.iloc[-1]], 'lime', linewidth=2)
ax.plot([shares25.asset2.iloc[-1], 0], [0, shares15.asset1.iloc[-1]], 'k--', linewidth=2, dashes=(3, 3))
ax.set_xlim(-5, shares15.asset1.max()+10)
for idx in range(s15.shape[0]):
    ax.plot(s25[idx], s15[idx], 'm', alpha=0.7)
for idx in range(os15.shape[0]):
    ax.plot(os25[idx], os15[idx], 'c', alpha=0.7)